# Plot exported data from the AKTA FPLC

Export curves from an AKTA chromatogram, then upload them to plot and save.

### Run the following cell to initialize script, upload files, and select one to plot.

Note that the uploaded files are in the AKTAfiles subdirectory. If you manually upload, they need to be placed in the ATKAfiles subdirectory.

In [1]:
# Initialize script
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import glob
import ipywidgets as widgets

file_uploader = widgets.FileUpload(multiple = True)

def on_file_upload(change):
    global select_file, current_file
    
    for details_dict in change['new']:
        if details_dict['name'][-3:] == 'asc'or details_dict['name'][-3:] == 'txt':
            NewFileName = './AKTAfiles/' + details_dict['name']
            files.append(NewFileName)
            file_list.append(details_dict['name'])
        with open(NewFileName, "wb") as f:
            f.write(details_dict['content'])

    current_file = './AKTAfiles/' + select_file.value
    select_file.options=tuple(file_list)


def on_select_file(change):
    global current_file
    current_file = './AKTAfiles/' + change['new']

from pathlib import Path
Path('./AKTAfiles').mkdir(parents=True, exist_ok = True)

files = glob.glob("./AKTAfiles/*.asc") + glob.glob("./AKTAfiles/*.txt")
file_list = []
for file in files:
    file_list.append(file[12:])

current_file = None

select_markdown = widgets.HTML(value="Use the <b>Upload</b> button to "
                               "upload your files if they are not among the choices you see "
                               "listed below.<br/>Click on the file name in the list below "
                               "to select the <b>AKTA file</b> to plot. "
                               " <br>Run the the next cell to proceed. <br> <br>"
                               " You can change selection without running this cell again. <br> <br>")


select_file = widgets.Select(options = file_list, description = 'AKTA runs:', layout={'width':'600px'})

#current_file = './AKTAfiles/' + select_file.value

hbox = widgets.HBox([file_uploader, select_file])
vbox = widgets.VBox([select_markdown, hbox])

display(vbox)

file_uploader.observe(on_file_upload, 'value')
select_file.observe(on_select_file, 'value')

def plot_elution(
          filename,
          B_elution=False,
          graphTitle='output',
          savefile=False,
          savescale=2,
          bgcolor='AliceBlue'
                                ):

    df = pd.read_table(filename, header=None, skiprows=1)#, usecols=range(ncols))
    df = df[list(range(0,df.count(axis='columns').max()))]
    index1 = list(filter(lambda x: x ==x, df.iloc[0].tolist()))
    new_index1 = []
    for item in index1:
        new_index1.append(item)
        new_index1.append(item)
    index2 = list(filter(lambda x: x ==x, df.iloc[1].tolist()))
    
    arrays = [new_index1,index2]
    multi_index =pd.MultiIndex.from_arrays(arrays,names=['channel','xy'])
    
    df = df.iloc[2:]
    df.columns = multi_index
    df = df.astype(float)
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    plot_list = []
    for plot in index1:
        if plot[:2] == 'UV':
            fig.add_trace(
                go.Scatter(x=df[plot,"ml"], y=df[plot,"mAU"], name= plot[:3] + plot[-3:]),
                                secondary_y=False)
            plot_list.append(plot[:3] + plot[-3:])
        elif plot[:4] == 'Conc':
            # Use add_trace function and specify secondary_y axes = True.
            B_elution == True
            fig.add_trace(
                go.Scatter(x=df[plot,"ml"], y=df[plot,"%"], name=plot),
                secondary_y=True,)
            plot_list.append(plot)
            
    fig.update_layout(
                        title_text=graphTitle,
                        template = 'simple_white',
                        height = 400,
                        width = 700,
                        xaxis = dict(title_text="volume (mL)",
                                     gridcolor='light gray',
                                     gridwidth = 0.3,
                                     showgrid = True,
                                     title_font_size = 16, 
                                     mirror = True),
                        yaxis = dict(title_text = "Abs (mAU)",
                                     gridcolor='light gray',
                                     gridwidth = 0.3,
                                     showgrid = True,
                                     title_font_size = 16, 
                                     mirror = True),
                        paper_bgcolor=bgcolor
                     )
    if B_elution:
        fig.update_yaxes(title_text="Conc B (%)", secondary_y=True)

    if savefile:  
            import plotly.io as pio
            pio.write_image(fig, graphTitle+'.png', scale = savescale)
    return plot_list, fig

# Update and save figure buttons


def save_plotly(b):
    global save_plotly_name, save_plotly_file_type, g
    
    from pathlib import Path
    Path('./Saves').mkdir(parents=True, exist_ok = True)
    
    file_name = './Saves/' + save_plotly_name.value.rstrip().lstrip()
    if (file_name[-4:] in ['.pdf','.png']):
        file_name = file_name[:-4] + '.' + save_plotly_file_type.value
    elif (file_name[-5:] in ['.html']):
        file_name = file_name[:-5] + '.' + save_plotly_file_type.value
    else:
        file_name = file_name + '.' + save_plotly_file_type.value
    if (save_plotly_file_type.value == 'pdf') or (save_plotly_file_type.value == 'png'):
        g.write_image(file_name, scale = 4)
    elif save_plotly_file_type.value == 'html':
        g.write_html(file_name)
    print("Saved to: ",file_name)


background_color = widgets.Dropdown(
    options=['aliceblue', 'white', 'palegoldenrod', 'lightgrey', 'lightskyblue', 'lightgreen', 'honeydew'],
    value='white',
    description='Sheet Color:',
    disabled=False
)

save_plotly_button = widgets.Button(description="Save")
save_plotly_name = widgets.Text(
    value='AKTAplot',
    placeholder='name = *.pdf, *.png, *.jpeg',
    description='Name of file:',
    disabled=False
    )

save_plotly_file_type = widgets.RadioButtons(
    options=['pdf', 'png', 'html'],
    description='Image file type:',
    disabled=False
)
_style = widgets.HTML(
    "<style>.widget-radio-box {flex-direction: row}.widget-radio-box"
    " label{margin:10px; width: 100px}</style>",
    layout=widgets.Layout(display="none")
)

def save_plotly(b):
    global save_plotly_name, save_plotly_file_type, g
    
    from pathlib import Path
    Path('./Saves').mkdir(parents=True, exist_ok = True)
    
    file_name = './Saves/' + save_plotly_name.value.rstrip().lstrip()
    if (file_name[-4:] in ['.pdf','.png']):
        file_name = file_name[:-4] + '.' + save_plotly_file_type.value
    elif (file_name[-5:] in ['.html']):
        file_name = file_name[:-5] + '.' + save_plotly_file_type.value
    else:
        file_name = file_name + '.' + save_plotly_file_type.value
    if (save_plotly_file_type.value == 'pdf') or (save_plotly_file_type.value == 'png'):
        g.write_image(file_name, scale = 4)
    elif save_plotly_file_type.value == 'html':
        g.write_html(file_name)
    print("Saved to: ",file_name)

def update_background(change):
    global g, background_color
    g.layout['paper_bgcolor']= background_color.value


TypeError: can only concatenate str (not "NoneType") to str

### Run the following cell to plot the curves.

You can modify the plot and save the results. Note that the saved files are in the Saves subdirectory.

Double-click on names in the legend to show/hide that curve.

In [ ]:
# Plot curves
# import graph_objects from plotly package
filename = current_file

if filename != None:
    title = filename[12:-4]
    plot_list, fig = plot_elution(filename,
                graphTitle=title,
                savefile=False,
                bgcolor='white'
                )

    g = go.FigureWidget(fig)

    vbox=widgets.VBox([background_color, save_plotly_name, save_plotly_file_type, _style, save_plotly_button])
    display(g,vbox)

else:
    print("Please select a file before running this cell.")


save_plotly_button.on_click(save_plotly)
background_color.observe(update_background, names = "value")